# Notebook 7: Feedback Generation

## Objective
Generate actionable, specific feedback based on resume scores to help users improve their resumes.

## Goals
1. Generate feedback based on component scores
2. Identify missing sections and requirements
3. Suggest skill improvements
4. Compare resume against job description (skill gaps)
5. Create prioritized improvement recommendations
6. Format feedback for Streamlit UI display

## Input
Using scoring results from Notebook 6

## Output
Structured feedback with:
- **Strengths** - What's good about the resume
- **Critical Issues** - Must-fix problems
- **Warnings** - Important improvements
- **Suggestions** - Nice-to-have enhancements
- **Skills Gap Analysis** - Missing skills vs job description


---


## 1. Setup and Imports


In [13]:
import re
from pathlib import Path
from typing import List, Dict

print("✓ Imports successful")
print("\nFeedback Generation System Ready")


✓ Imports successful

Feedback Generation System Ready


---


## 2. Feedback Generation Functions

Create functions to analyze each score component and generate specific recommendations.


In [14]:
def generate_completeness_feedback(details: dict) -> dict:
    """Generate feedback for completeness component."""
    feedback = {'strengths': [], 'warnings': [], 'critical': []}
    
    found = details.get('found_sections', [])
    missing = details.get('missing_required', [])
    
    if len(found) >= 5:
        feedback['strengths'].append('✅ Complete resume with all essential sections')
    elif len(found) >= 3:
        feedback['strengths'].append('✅ Has core sections')
    
    for section in missing:
        feedback['critical'].append(f'🚨 CRITICAL: Missing {section.upper()} section')
    
    # Check for optional beneficial sections
    if 'summary' not in found:
        feedback['warnings'].append('⚠️ Add SUMMARY/OBJECTIVE section for better first impression')
    
    if 'certifications' not in found:
        feedback['warnings'].append('💡 Consider adding CERTIFICATIONS section if applicable')
    
    if 'projects' not in found:
        feedback['warnings'].append('💡 Add PROJECTS section to showcase practical work')
    
    return feedback


# Test
test_comp_details = {
    'found_sections': ['experience', 'education', 'skills', 'summary'],
    'missing_required': [],
    'section_count': 4
}

comp_feedback = generate_completeness_feedback(test_comp_details)
print("Completeness Feedback Example:")
print("  Strengths:", comp_feedback['strengths'])
print("  Warnings:", comp_feedback['warnings'][:2])
print("✓ Function defined")


Completeness Feedback Example:
  Strengths: ['✅ Has core sections']
  Warnings: ['💡 Consider adding CERTIFICATIONS section if applicable', '💡 Add PROJECTS section to showcase practical work']
✓ Function defined


In [15]:
def generate_skills_feedback(details: dict) -> dict:
    """Generate feedback for skills component."""
    feedback = {'strengths': [], 'warnings': [], 'suggestions': [], 'missing_skills': []}
    
    skill_count = details.get('resume_skill_count', 0)
    
    if skill_count >= 15:
        feedback['strengths'].append(f'✅ Excellent skill diversity ({skill_count} skills found)')
    elif skill_count >= 8:
        feedback['strengths'].append(f'✅ Good skill set ({skill_count} skills)')
    elif skill_count >= 5:
        feedback['warnings'].append(f'⚠️ Moderate skills ({skill_count}). Aim for 10+ skills')
    else:
        feedback['warnings'].append(f'⚠️ Limited skills ({skill_count}). Add more relevant technologies')
    
    # Job matching feedback
    if 'job_skills' in details:
        match_pct = details.get('match_percentage', 0)
        matching = details.get('matching_skills', [])
        missing = set(details.get('job_skills', [])) - set(matching)
        
        if match_pct >= 80:
            feedback['strengths'].append(f'✅ Excellent job match ({match_pct:.0f}% of required skills)')
        elif match_pct >= 60:
            feedback['strengths'].append(f'✅ Good job match ({match_pct:.0f}%)')
        elif match_pct >= 40:
            feedback['warnings'].append(f'⚠️ Fair job match ({match_pct:.0f}%). Review requirements')
        else:
            feedback['warnings'].append(f'⚠️ Low job match ({match_pct:.0f}%). Missing key skills')
        
        # List missing critical skills
        for skill in list(missing)[:5]:
            feedback['missing_skills'].append(f'❌ Missing: {skill}')
            feedback['suggestions'].append(f'💡 Add "{skill}" to skills section')
    
    return feedback


# Test
test_skills_details = {
    'resume_skill_count': 12,
    'job_skills': ['Python', 'AWS', 'Docker', 'Kubernetes', 'React'],
    'matching_skills': ['Python', 'AWS', 'Docker'],
    'match_percentage': 60
}

skills_feedback = generate_skills_feedback(test_skills_details)
print("\nSkills Feedback Example:")
print("  Strengths:", skills_feedback['strengths'])
print("  Missing:", skills_feedback['missing_skills'])
print("  Suggestions:", skills_feedback['suggestions'][:2])
print("✓ Function defined")



Skills Feedback Example:
  Strengths: ['✅ Good skill set (12 skills)', '✅ Good job match (60%)']
  Missing: ['❌ Missing: Kubernetes', '❌ Missing: React']
  Suggestions: ['💡 Add "Kubernetes" to skills section', '💡 Add "React" to skills section']
✓ Function defined


In [16]:
def generate_experience_feedback(details: dict) -> dict:
    """Generate feedback for experience component."""
    feedback = {'strengths': [], 'warnings': [], 'suggestions': []}
    
    years = details.get('years', 0)
    quality_indicators = details.get('quality_indicators', [])
    
    # Years feedback
    if years >= 8:
        feedback['strengths'].append(f'✅ Extensive experience ({years}+ years)')
    elif years >= 5:
        feedback['strengths'].append(f'✅ Strong experience ({years} years)')
    elif years >= 2:
        feedback['strengths'].append(f'✅ Relevant experience ({years} years)')
    elif years > 0:
        feedback['warnings'].append(f'⚠️ Limited experience ({years} year(s)). Highlight relevant projects')
    else:
        feedback['warnings'].append('🚨 No years of experience mentioned. Add dates to positions')
    
    # Quality indicators feedback
    if 'Leadership' in quality_indicators:
        feedback['strengths'].append('✅ Demonstrates leadership experience')
    else:
        feedback['suggestions'].append('💡 Mention leadership/management experience if applicable')
    
    if 'Senior Level' in quality_indicators:
        feedback['strengths'].append('✅ Senior-level position')
    
    if 'Quantified Achievements' in quality_indicators:
        feedback['strengths'].append('✅ Includes measurable achievements (%, $, numbers)')
    else:
        feedback['warnings'].append('⚠️ Add quantified achievements (improved X by Y%, managed $Z)')
    
    if 'Multiple Positions' in quality_indicators:
        feedback['strengths'].append('✅ Shows career progression')
    else:
        feedback['suggestions'].append('💡 List multiple positions to show growth')
    
    return feedback


# Test
test_exp_details = {
    'years': 6,
    'quality_indicators': ['Leadership', 'Senior Level', 'Quantified Achievements']
}

exp_feedback = generate_experience_feedback(test_exp_details)
print("\nExperience Feedback Example:")
print("  Strengths:", exp_feedback['strengths'][:3])
print("✓ Function defined")



Experience Feedback Example:
  Strengths: ['✅ Strong experience (6 years)', '✅ Demonstrates leadership experience', '✅ Senior-level position']
✓ Function defined


In [17]:
def generate_education_feedback(details: dict) -> dict:
    """Generate feedback for education component."""
    feedback = {'strengths': [], 'warnings': [], 'suggestions': []}
    
    highest = details.get('highest_degree')
    prestigious = details.get('prestigious_institution', False)
    institutions = details.get('institutions_found', [])
    
    if highest:
        if 'PhD' in highest or 'Doctorate' in highest:
            feedback['strengths'].append('✅ Doctoral degree (PhD)')
        elif 'Master' in highest or 'MBA' in highest:
            feedback['strengths'].append('✅ Advanced degree (Master\'s/MBA)')
        elif 'Bachelor' in highest:
            feedback['strengths'].append('✅ Bachelor\'s degree')
        else:
            feedback['warnings'].append('⚠️ Specify complete degree name (B.S., M.S., etc.)')
    else:
        feedback['warnings'].append('🚨 No degree found. Clearly state your education level')
    
    if prestigious:
        feedback['strengths'].append(f'✅ Top-tier institution ({institutions[0]})')
    
    return feedback


# Test
test_edu_details = {
    'highest_degree': 'Master',
    'prestigious_institution': True,
    'institutions_found': ['MIT']
}

edu_feedback = generate_education_feedback(test_edu_details)
print("\nEducation Feedback Example:")
print("  Strengths:", edu_feedback['strengths'])
print("✓ Function defined")



Education Feedback Example:
  Strengths: ["✅ Advanced degree (Master's/MBA)", '✅ Top-tier institution (MIT)']
✓ Function defined


In [18]:
def generate_format_feedback(details: dict) -> dict:
    """Generate feedback for format quality."""
    feedback = {'strengths': [], 'warnings': [], 'suggestions': []}
    
    good_practices = details.get('good_practices', [])
    issues = details.get('issues', [])
    word_count = details.get('word_count', 0)
    
    for practice in good_practices:
        if 'ATS-friendly' in practice:
            feedback['strengths'].append('✅ ATS-friendly format (minimal special characters)')
        elif 'bullet points' in practice:
            feedback['strengths'].append('✅ Uses bullet points for readability')
        elif 'email' in practice:
            feedback['strengths'].append('✅ Contact information included')
        elif 'header' in practice:
            feedback['strengths'].append('✅ Clear section headers')
    
    for issue in issues:
        if 'Too short' in issue:
            feedback['warnings'].append(f'⚠️ Resume too short ({word_count} words). Aim for 600-800 words')
        elif 'Too long' in issue:
            feedback['warnings'].append(f'⚠️ Resume too long ({word_count} words). Keep under 900 words')
        elif 'email' in issue:
            feedback['warnings'].append('🚨 CRITICAL: Missing email address')
        elif 'phone' in issue:
            feedback['suggestions'].append('💡 Add phone number')
        elif 'header' in issue:
            feedback['warnings'].append('⚠️ Use clear ALL CAPS section headers (EXPERIENCE, EDUCATION, etc.)')
        elif 'special characters' in issue:
            feedback['warnings'].append('⚠️ Too many special characters. Simplify formatting for ATS')
    
    return feedback


# Test
test_format_details = {
    'word_count': 450,
    'good_practices': ['Has email', 'Uses bullet points', 'ATS-friendly (minimal special chars)'],
    'issues': ['Too short (450 words)']
}

format_feedback = generate_format_feedback(test_format_details)
print("\nFormat Feedback Example:")
print("  Strengths:", format_feedback['strengths'][:2])
print("  Warnings:", format_feedback['warnings'])
print("✓ Function defined")



Format Feedback Example:
  Strengths: ['✅ Contact information included', '✅ Uses bullet points for readability']
  Warnings: ['⚠️ Resume too short (450 words). Aim for 600-800 words']
✓ Function defined


---


## 3. Complete Feedback Generator

Combine all component feedback into a comprehensive recommendation system.


In [19]:
def generate_comprehensive_feedback(score_result: dict) -> dict:
    """
    Generate complete feedback for all components.
    
    Args:
        score_result: Result from calculate_resume_score()
    
    Returns:
        Dictionary with categorized feedback
    """
    components = score_result['components']
    
    # Generate feedback for each component
    comp_fb = generate_completeness_feedback(components['completeness']['details'])
    skills_fb = generate_skills_feedback(components['skills']['details'])
    exp_fb = generate_experience_feedback(components['experience']['details'])
    edu_fb = generate_education_feedback(components['education']['details'])
    format_fb = generate_format_feedback(components['format_quality']['details'])
    
    # Combine all feedback
    feedback = {
        'overall_score': score_result['total_score'],
        'grade': score_result['grade'],
        'strengths': [],
        'critical_issues': [],
        'warnings': [],
        'suggestions': [],
        'missing_skills': skills_fb.get('missing_skills', []),
        'priority_actions': []
    }
    
    # Combine strengths
    feedback['strengths'].extend(comp_fb['strengths'])
    feedback['strengths'].extend(skills_fb['strengths'])
    feedback['strengths'].extend(exp_fb['strengths'])
    feedback['strengths'].extend(edu_fb['strengths'])
    feedback['strengths'].extend(format_fb['strengths'])
    
    # Combine critical issues
    feedback['critical_issues'].extend(comp_fb.get('critical', []))
    feedback['critical_issues'].extend(skills_fb.get('critical', []))
    feedback['critical_issues'].extend(exp_fb.get('critical', []))
    feedback['critical_issues'].extend(edu_fb.get('critical', []))
    
    # Combine warnings
    feedback['warnings'].extend(comp_fb['warnings'])
    feedback['warnings'].extend(skills_fb['warnings'])
    feedback['warnings'].extend(exp_fb['warnings'])
    feedback['warnings'].extend(edu_fb['warnings'])
    feedback['warnings'].extend(format_fb['warnings'])
    
    # Combine suggestions
    feedback['suggestions'].extend(comp_fb.get('suggestions', []))
    feedback['suggestions'].extend(skills_fb['suggestions'])
    feedback['suggestions'].extend(exp_fb['suggestions'])
    feedback['suggestions'].extend(edu_fb['suggestions'])
    feedback['suggestions'].extend(format_fb['suggestions'])
    
    # Generate priority actions (top 3 most important)
    if feedback['critical_issues']:
        feedback['priority_actions'] = feedback['critical_issues'][:3]
    elif feedback['warnings']:
        feedback['priority_actions'] = feedback['warnings'][:3]
    else:
        feedback['priority_actions'] = feedback['suggestions'][:3]
    
    return feedback


print("✓ Complete feedback generation function defined")


✓ Complete feedback generation function defined


---


## 4. Test Complete Feedback System

Using the scoring functions from Notebook 6 (simulated here for testing).


In [20]:
# Simulate a score result (in production, this comes from Notebook 6)
test_score_result = {
    'total_score': 73.5,
    'grade': 'B (Good)',
    'components': {
        'completeness': {
            'score': 17,
            'max': 20,
            'details': {
                'found_sections': ['experience', 'education', 'skills', 'summary'],
                'missing_required': [],
                'section_count': 4
            }
        },
        'skills': {
            'score': 17.2,
            'max': 25,
            'details': {
                'resume_skill_count': 12,
                'job_skills': ['Python', 'AWS', 'Docker', 'Kubernetes', 'React', 'SQL'],
                'matching_skills': ['Python', 'AWS', 'Docker', 'React'],
                'match_percentage': 66.7
            }
        },
        'experience': {
            'score': 15,
            'max': 20,
            'details': {
                'years': 5,
                'quality_indicators': ['Senior Level', 'Quantified Achievements']
            }
        },
        'education': {
            'score': 12,
            'max': 15,
            'details': {
                'highest_degree': 'Master',
                'prestigious_institution': False,
                'institutions_found': []
            }
        },
        'certifications': {
            'score': 4,
            'max': 10,
            'details': {
                'found_certifications': ['AWS', 'PMP'],
                'certification_count': 2
            }
        },
        'format_quality': {
            'score': 8.3,
            'max': 10,
            'details': {
                'word_count': 520,
                'good_practices': ['Has email', 'Has phone', 'Clear section headers (4)', 'Uses bullet points'],
                'issues': []
            }
        }
    }
}

# Generate feedback
feedback = generate_comprehensive_feedback(test_score_result)

print("Complete Feedback Generation Test")
print("="*80)
print(f"\n📊 OVERALL SCORE: {feedback['overall_score']:.1f}/100")
print(f"📈 GRADE: {feedback['grade']}")

print("\n🌟 STRENGTHS:")
for strength in feedback['strengths'][:5]:
    print(f"  {strength}")

if feedback['critical_issues']:
    print("\n🚨 CRITICAL ISSUES:")
    for issue in feedback['critical_issues']:
        print(f"  {issue}")

if feedback['warnings']:
    print("\n⚠️  WARNINGS:")
    for warning in feedback['warnings'][:4]:
        print(f"  {warning}")

if feedback['missing_skills']:
    print("\n❌ MISSING SKILLS (vs Job):")
    for skill in feedback['missing_skills']:
        print(f"  {skill}")

if feedback['suggestions']:
    print("\n💡 SUGGESTIONS:")
    for suggestion in feedback['suggestions'][:4]:
        print(f"  {suggestion}")

print("\n🎯 TOP PRIORITY ACTIONS:")
for action in feedback['priority_actions'][:3]:
    print(f"  {action}")

print("\n" + "="*80)
print("✓ Comprehensive feedback generated successfully!")


Complete Feedback Generation Test

📊 OVERALL SCORE: 73.5/100
📈 GRADE: B (Good)

🌟 STRENGTHS:
  ✅ Has core sections
  ✅ Good skill set (12 skills)
  ✅ Good job match (67%)
  ✅ Strong experience (5 years)
  ✅ Senior-level position

⚠️  WARNINGS:
  💡 Consider adding CERTIFICATIONS section if applicable
  💡 Add PROJECTS section to showcase practical work

❌ MISSING SKILLS (vs Job):
  ❌ Missing: Kubernetes
  ❌ Missing: SQL

💡 SUGGESTIONS:
  💡 Add "Kubernetes" to skills section
  💡 Add "SQL" to skills section
  💡 Mention leadership/management experience if applicable
  💡 List multiple positions to show growth

🎯 TOP PRIORITY ACTIONS:
  💡 Consider adding CERTIFICATIONS section if applicable
  💡 Add PROJECTS section to showcase practical work

✓ Comprehensive feedback generated successfully!


---


## 5. Format Feedback for Display

Create formatted output suitable for Streamlit UI.


In [21]:
def format_feedback_for_display(feedback: dict) -> str:
    """
    Format feedback as readable text for UI display.
    
    Args:
        feedback: Feedback dictionary from generate_comprehensive_feedback()
    
    Returns:
        Formatted string ready for display
    """
    output = []
    
    # Header
    output.append("="*80)
    output.append(f"📊 RESUME ANALYSIS REPORT")
    output.append("="*80)
    output.append(f"\n🎯 Overall Score: {feedback['overall_score']:.1f}/100")
    output.append(f"📈 Grade: {feedback['grade']}")
    output.append("")
    
    # Critical Issues (if any)
    if feedback['critical_issues']:
        output.append("🚨 CRITICAL ISSUES - Fix These First!")
        output.append("-"*80)
        for issue in feedback['critical_issues']:
            output.append(f"  {issue}")
        output.append("")
    
    # Priority Actions
    if feedback['priority_actions']:
        output.append("🎯 TOP 3 PRIORITY ACTIONS:")
        output.append("-"*80)
        for i, action in enumerate(feedback['priority_actions'][:3], 1):
            output.append(f"  {i}. {action}")
        output.append("")
    
    # Missing Skills
    if feedback['missing_skills']:
        output.append("❌ SKILL GAPS (vs Job Description):")
        output.append("-"*80)
        for skill in feedback['missing_skills'][:5]:
            output.append(f"  {skill}")
        output.append("")
    
    # Warnings
    if feedback['warnings']:
        output.append("⚠️  IMPORTANT IMPROVEMENTS:")
        output.append("-"*80)
        for warning in feedback['warnings'][:5]:
            output.append(f"  {warning}")
        output.append("")
    
    # Strengths
    if feedback['strengths']:
        output.append("🌟 STRENGTHS:")
        output.append("-"*80)
        for strength in feedback['strengths'][:8]:
            output.append(f"  {strength}")
        output.append("")
    
    # Suggestions
    if feedback['suggestions']:
        output.append("💡 ADDITIONAL SUGGESTIONS:")
        output.append("-"*80)
        for suggestion in feedback['suggestions'][:5]:
            output.append(f"  {suggestion}")
        output.append("")
    
    output.append("="*80)
    
    return '\n'.join(output)


# Test formatted output
formatted_output = format_feedback_for_display(feedback)

print("Formatted Feedback Display:")
print(formatted_output)
print("\n✓ Formatting function defined")


Formatted Feedback Display:
📊 RESUME ANALYSIS REPORT

🎯 Overall Score: 73.5/100
📈 Grade: B (Good)

🎯 TOP 3 PRIORITY ACTIONS:
--------------------------------------------------------------------------------
  1. 💡 Consider adding CERTIFICATIONS section if applicable
  2. 💡 Add PROJECTS section to showcase practical work

❌ SKILL GAPS (vs Job Description):
--------------------------------------------------------------------------------
  ❌ Missing: Kubernetes
  ❌ Missing: SQL

⚠️  IMPORTANT IMPROVEMENTS:
--------------------------------------------------------------------------------
  💡 Consider adding CERTIFICATIONS section if applicable
  💡 Add PROJECTS section to showcase practical work

🌟 STRENGTHS:
--------------------------------------------------------------------------------
  ✅ Has core sections
  ✅ Good skill set (12 skills)
  ✅ Good job match (67%)
  ✅ Strong experience (5 years)
  ✅ Senior-level position
  ✅ Includes measurable achievements (%, $, numbers)
  ✅ Advanced degre

---


## 6. Test Different Resume Scenarios

Test feedback generation for various resume quality levels.


In [22]:
# Test Case 1: Weak Resume
weak_resume_score = {
    'total_score': 35,
    'grade': 'F (Poor)',
    'components': {
        'completeness': {
            'score': 10,
            'details': {
                'found_sections': ['experience', 'education'],
                'missing_required': ['skills'],
                'section_count': 2
            }
        },
        'skills': {
            'score': 5,
            'details': {
                'resume_skill_count': 3,
                'job_skills': ['Python', 'AWS', 'Docker'],
                'matching_skills': [],
                'match_percentage': 0
            }
        },
        'experience': {
            'score': 7,
            'details': {
                'years': 2,
                'quality_indicators': []
            }
        },
        'education': {
            'score': 9,
            'details': {
                'highest_degree': 'Bachelor'
            }
        },
        'certifications': {
            'score': 0,
            'details': {'certification_count': 0}
        },
        'format_quality': {
            'score': 4,
            'details': {
                'word_count': 280,
                'good_practices': [],
                'issues': ['Too short (280 words)', 'Weak section headers']
            }
        }
    }
}

weak_feedback = generate_comprehensive_feedback(weak_resume_score)

print("WEAK RESUME FEEDBACK:")
print("="*80)
print(f"Score: {weak_feedback['overall_score']}/100 - {weak_feedback['grade']}")
print(f"\nCritical Issues: {len(weak_feedback['critical_issues'])}")
print(f"Warnings: {len(weak_feedback['warnings'])}")
print("\nTop Priority:")
for action in weak_feedback['priority_actions'][:3]:
    print(f"  {action}")
print("="*80)


WEAK RESUME FEEDBACK:
Score: 35/100 - F (Poor)

Critical Issues: 1
Warnings: 8

Top Priority:
  🚨 CRITICAL: Missing SKILLS section


---


## 7. Production Code

Final production-ready feedback generator class.


In [23]:
# PRODUCTION CODE

class FeedbackGenerator:
    """
    Generate actionable feedback based on resume scores.
    """
    
    def generate(self, score_result: dict) -> dict:
        """
        Generate comprehensive feedback from scoring results.
        
        Args:
            score_result: Dictionary from ResumeScorer.score()
        
        Returns:
            Dictionary with categorized feedback
        """
        components = score_result['components']
        
        feedback = {
            'overall_score': score_result['total_score'],
            'grade': score_result['grade'],
            'strengths': [],
            'critical_issues': [],
            'warnings': [],
            'suggestions': [],
            'missing_skills': [],
            'priority_actions': []
        }
        
        # Generate component-specific feedback
        self._add_completeness_feedback(feedback, components['completeness']['details'])
        self._add_skills_feedback(feedback, components['skills']['details'])
        self._add_experience_feedback(feedback, components['experience']['details'])
        self._add_education_feedback(feedback, components['education']['details'])
        self._add_format_feedback(feedback, components['format_quality']['details'])
        
        # Determine priority actions
        if feedback['critical_issues']:
            feedback['priority_actions'] = feedback['critical_issues'][:3]
        elif feedback['warnings']:
            feedback['priority_actions'] = feedback['warnings'][:3]
        else:
            feedback['priority_actions'] = feedback['suggestions'][:3]
        
        return feedback
    
    def _add_completeness_feedback(self, feedback: dict, details: dict):
        """Add completeness feedback."""
        found = len(details.get('found_sections', []))
        missing = details.get('missing_required', [])
        
        if found >= 5:
            feedback['strengths'].append('✅ Complete resume structure')
        
        for section in missing:
            feedback['critical_issues'].append(f'🚨 Missing {section.upper()} section')
    
    def _add_skills_feedback(self, feedback: dict, details: dict):
        """Add skills feedback."""
        count = details.get('resume_skill_count', 0)
        
        if count >= 10:
            feedback['strengths'].append(f'✅ Strong skill set ({count} skills)')
        elif count < 5:
            feedback['warnings'].append(f'⚠️ Limited skills. Add more technologies')
        
        # Job matching
        if 'job_skills' in details:
            missing = set(details.get('job_skills', [])) - set(details.get('matching_skills', []))
            for skill in list(missing)[:3]:
                feedback['missing_skills'].append(skill)
                feedback['suggestions'].append(f'💡 Add "{skill}"')
    
    def _add_experience_feedback(self, feedback: dict, details: dict):
        """Add experience feedback."""
        years = details.get('years', 0)
        
        if years >= 5:
            feedback['strengths'].append(f'✅ Strong experience ({years} years)')
        elif years == 0:
            feedback['warnings'].append('⚠️ Specify years of experience')
        
        if 'Quantified Achievements' not in details.get('quality_indicators', []):
            feedback['warnings'].append('⚠️ Add measurable achievements')
    
    def _add_education_feedback(self, feedback: dict, details: dict):
        """Add education feedback."""
        degree = details.get('highest_degree')
        
        if degree and ('Master' in degree or 'PhD' in degree):
            feedback['strengths'].append('✅ Advanced degree')
        elif not degree:
            feedback['warnings'].append('⚠️ Specify education level')
    
    def _add_format_feedback(self, feedback: dict, details: dict):
        """Add format feedback."""
        wc = details.get('word_count', 0)
        
        if 'ATS-friendly' in str(details.get('good_practices', [])):
            feedback['strengths'].append('✅ ATS-friendly format')
        
        if wc < 500:
            feedback['warnings'].append(f'⚠️ Too short ({wc} words). Aim for 600+')
    
    def format_for_display(self, feedback: dict) -> str:
        """Format feedback as readable text."""
        lines = [
            "="*80,
            "📊 RESUME ANALYSIS",
            "="*80,
            f"\n🎯 Score: {feedback['overall_score']:.1f}/100 - {feedback['grade']}\n"
        ]
        
        if feedback['priority_actions']:
            lines.append("\n🎯 TOP PRIORITIES:")
            for i, action in enumerate(feedback['priority_actions'][:3], 1):
                lines.append(f"  {i}. {action}")
        
        if feedback['missing_skills']:
            lines.append("\n❌ MISSING SKILLS:")
            for skill in feedback['missing_skills'][:3]:
                lines.append(f"  • {skill}")
        
        if feedback['strengths']:
            lines.append("\n🌟 STRENGTHS:")
            for strength in feedback['strengths'][:5]:
                lines.append(f"  {strength}")
        
        lines.append("\n" + "="*80)
        return '\n'.join(lines)


print("✓ Production class defined: FeedbackGenerator")
print("\nMethods:")
print("  - generate(score_result)")
print("  - format_for_display(feedback)")
print("\nReady for models/feedback_generator.py")


✓ Production class defined: FeedbackGenerator

Methods:
  - generate(score_result)
  - format_for_display(feedback)

Ready for models/feedback_generator.py


In [24]:
# Test production class
generator = FeedbackGenerator()
prod_feedback = generator.generate(test_score_result)
formatted_text = generator.format_for_display(prod_feedback)

print("Production Class Test:")
print(formatted_text)
print("\n✓ Production class tested successfully")


Production Class Test:
📊 RESUME ANALYSIS

🎯 Score: 73.5/100 - B (Good)


🎯 TOP PRIORITIES:
  1. 💡 Add "Kubernetes"
  2. 💡 Add "SQL"

❌ MISSING SKILLS:
  • Kubernetes
  • SQL

🌟 STRENGTHS:
  ✅ Strong skill set (12 skills)
  ✅ Strong experience (5 years)
  ✅ Advanced degree


✓ Production class tested successfully
